<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning-LANA/blob/master/tensorflow/TensorFlow_RegressaoLinear_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regressão Linear com TensorFlow - versão 2

## Carregando o modelo

In [1]:
!wget https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/tensorflow/data/vendas_data_training.csv
!wget https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/tensorflow/data/vendas_data_test.csv
!mkdir datasets
!mv *.csv datasets
!ls -ilah datasets

--2019-01-25 15:15:40--  https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/tensorflow/data/vendas_data_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30718 (30K) [text/plain]
Saving to: ‘vendas_data_training.csv’

vendas_data_trainin 100%[===================>]  30.00K  --.-KB/s    in 0.007s  

2019-01-25 15:15:40 (4.24 MB/s) - ‘vendas_data_training.csv’ saved [30718/30718]

--2019-01-25 15:15:42--  https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/tensorflow/data/vendas_data_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.


In [2]:
# Versão 2 do Modelo

import os
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Nível de log
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'

# ------------------------- Seção 1 - Carregando e Normalizando os Dados -------------------------

# Carrega dados de treino
dados_treino = pd.read_csv("datasets/vendas_data_training.csv", dtype = float)

# Define X e Y de treino
X_treino = dados_treino.drop("total_vendas", axis= 1).values
Y_treino = dados_treino[['total_vendas']].values

# Carrega dados de teste
dados_teste = pd.read_csv("datasets/vendas_data_test.csv", dtype = float)

# Define X e Y de teste
X_teste = dados_teste.drop("total_vendas", axis = 1).values
Y_teste = dados_teste[['total_vendas']].values

# Criando operadores de escala
X_scaler = MinMaxScaler(feature_range=(0, 1))
Y_scaler = MinMaxScaler(feature_range=(0, 1))

# Aplicando escala aos dados de treino
X_scaled_treino = X_scaler.fit_transform(X_treino)
Y_scaled_treino = Y_scaler.fit_transform(Y_treino)

# Aplicando escala aos dados de teste
X_scaled_teste = X_scaler.transform(X_teste)
Y_scaled_teste = Y_scaler.transform(Y_teste)

# ------------------------- Seção 2 - Estrutura do Modelo -------------------------

# Hiperparâmetros
learning_rate = 0.001
num_epochs = 100
display_step = 5

# Definindo inputs e outputs
num_inputs = 9
num_outputs = 1

# Camadas
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

# Run Tensorboard
RUN_NAME = "Histograma"

# ------------------------- Seção 3 - Construindo Camadas da Rede Neural -------------------------

# Camada de Input
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, num_inputs))

# Camada 1
with tf.variable_scope('layer1'):
    weights = tf.get_variable(name = 'weights1', shape=[num_inputs, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name = 'biases1', shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

# Camada 2
with tf.variable_scope('layer2'):
    weights = tf.get_variable(name = 'weights2', shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name = 'biases2', shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

# Camada 3
with tf.variable_scope('layer3'):
    weights = tf.get_variable(name = 'weights3', shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name = 'biases3', shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

# Camada de Output
with tf.variable_scope('output'):
    weights = tf.get_variable(name = 'weights4', shape=[layer_3_nodes, num_outputs], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name = 'biases4', shape=[num_outputs], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases

# ------------------------- Seção 4 - Custo e Otimização -------------------------

# Custo
with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, num_outputs))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

# Otimizador
with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# ------------------------- Seção 5 - Summary TensorBoard -------------------------

# Summary
with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    tf.summary.histogram('predicted_value', prediction)
    summary = tf.summary.merge_all()

saver = tf.train.Saver()

# ------------------------- Seção 6 - Sessão TensorFlow -------------------------

# Abrindo Sessão TensorFlow
with tf.Session() as session:

    # Inicialização das Variáveis
    session.run(tf.global_variables_initializer())

    # File writer para tracking do progresso
    training_writer = tf.summary.FileWriter("./logs/{}/training".format(RUN_NAME), session.graph)
    testing_writer = tf.summary.FileWriter("./logs/{}/testing".format(RUN_NAME), session.graph)

    # Loop for pelas epochs
    for epoch in range(num_epochs):

        # Treinamento
        session.run(optimizer, feed_dict={X:X_scaled_treino, Y:Y_scaled_treino})

        # Progresso do Treinamento
        if epoch % display_step == 0:
            training_cost, training_summary = session.run([cost, summary], feed_dict={X:X_scaled_treino, Y:Y_scaled_treino})
            test_cost, test_summary = session.run([cost, summary], feed_dict={X: X_scaled_teste, Y: Y_scaled_teste})

            # Adicionando Sumário
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(test_summary, epoch)

    # Print
    print("\nTreinamento Concluído")

    final_training_cost = session.run(cost, feed_dict={X: X_scaled_treino, Y: Y_scaled_treino})
    final_test_cost = session.run(cost, feed_dict={X: X_scaled_teste, Y: Y_scaled_teste})

    print("\nCusto Final em Treinamento: {}".format(final_training_cost))
    print("Custo Final em Teste: {}".format(final_test_cost))

    # Variável com valores previstos normalizados
    Y_predicted_scaled = session.run(prediction, feed_dict={X:X_scaled_teste})

    # Remove a escala
    Y_predicted = Y_scaler.inverse_transform(Y_predicted_scaled)

    # Coleta os dados reais e os valores previstos
    total_vendas_real = dados_teste['total_vendas'].values[0]
    total_vendas_previsto = Y_predicted[0][0]

    # Print do resultado
    print("\nTotal Vendas Real de 1 Seguro: {}".format(total_vendas_real))
    print("Total Vendas Previsto de 1 Seguro: {}".format(total_vendas_previsto))

    # Salvando o modelo
    save_path = saver.save(session, "modelos/modelo_treinado_v2.ckpt") # checkpoint
    print("\nModelo salvo: {}".format(save_path))



Treinamento Concluído

Custo Final em Treinamento: 0.00017210739315487444
Custo Final em Teste: 0.00026958619127981365

Total Vendas Real de 1 Seguro: 247537.0
Total Vendas Previsto de 1 Seguro: 251262.796875

Modelo salvo: modelos/modelo_treinado_v2.ckpt


# Carregando o modelo a partir do checkpoint

In [3]:
# Abrindo Sessão TensorFlow
with tf.Session() as session:

    # Restaurando o modelo
    saver.restore(session, "modelos/modelo_treinado_v2.ckpt")
    print("\nModelo Restaurado")

    # File writer para tracking do progresso
    training_writer = tf.summary.FileWriter("./logs/{}/training".format(RUN_NAME), session.graph)
    testing_writer = tf.summary.FileWriter("./logs/{}/testing".format(RUN_NAME), session.graph)


    final_training_cost = session.run(cost, feed_dict={X: X_scaled_treino, Y: Y_scaled_treino})
    final_test_cost = session.run(cost, feed_dict={X: X_scaled_teste, Y: Y_scaled_teste})

    print("\nCusto Final em Treinamento: {}".format(final_training_cost))
    print("Custo Final em Teste: {}".format(final_test_cost))

    # Variável com valores previstos normalizados
    Y_predicted_scaled = session.run(prediction, feed_dict={X:X_scaled_teste})

    # Remove a escala
    Y_predicted = Y_scaler.inverse_transform(Y_predicted_scaled)

    # Coleta os dados reais e os valores previstos
    total_vendas_real = dados_teste['total_vendas'].values[0]
    total_vendas_previsto = Y_predicted[0][0]

    # Print do resultado
    print("\nTotal Vendas Real de 1 Seguro: {}".format(total_vendas_real))
    print("Total Vendas Previsto de 1 Seguro: {}".format(total_vendas_previsto))

    # Salvando o modelo
    # save_path = saver.save(session, "modelos/modelo_treinado_v2.ckpt")
    # print("\nModelo salvo: {}".format(save_path))


INFO:tensorflow:Restoring parameters from modelos/modelo_treinado_v2.ckpt

Modelo Restaurado

Custo Final em Treinamento: 0.00017210739315487444
Custo Final em Teste: 0.00026958619127981365

Total Vendas Real de 1 Seguro: 247537.0
Total Vendas Previsto de 1 Seguro: 251262.796875
